In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv

# from holoviews.operation.datashader import regrid
import skimage

# import skimage.filters
# import skimage.feature
# import scipy.ndimage
from tqdm.auto import tqdm
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose, juxt
from itertools import repeat
from glob import glob

# import cachetools

# import numpy_indexed
import pickle

# import pyarrow as pa
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Config

In [ ]:
# dask.config.config['distributed']['scheduler']['allowed-failures'] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = LocalCluster(scheduler_options=dict(dashboard_address="0.0.0.0:8787"))
client = Client(cluster)

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="00:30:00",
    memory="6GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/paulssonlab/paulssonlab/src/paulssonlab/projects/sigma_circuits/log",
    cores=1,
    processes=1,
    scheduler_options=dict(dashboard_address="0.0.0.0:8787"),
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
# cluster.scale(30)
cluster.adapt(maximum=100)

# Segmentation

In [ ]:
good_positions = (
    np.array([17, 25, 54, 64, 96, 124, 150]) - 1
)  # 1-indexing to 0-indexing

In [ ]:
good_positions2 = (good_positions + np.arange(4)[:, np.newaxis]).T.reshape(-1)

In [ ]:
good_positions2

In [ ]:
nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"

In [ ]:
%%time
data_graph = segmentation.process(
    nd2_filename, time_slice=slice(None), position_slice=good_positions2, delayed=True
)

In [ ]:
%%time
data_future = client.compute(data_graph)

In [ ]:
%%time
data = client.gather(data_future)

In [ ]:
data.info(memory_usage="full")

## Save data

In [ ]:
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329flippedpwm3.pickle"
with open(filename, "wb") as f:
    pickle.dump(data, f)